## Web Crawler de criticas de filmes

In [1]:
from bs4 import BeautifulSoup
from bs4.dammit import EncodingDetector
import urllib.robotparser as robotparser
from urllib.request import urlopen
import requests as rq
from collections import deque
import re
import time
import random
from random import randrange
import json
import ssl


### Paginas

In [2]:
paginas = [ ('Pocilga','pocilga.com.br/'), ('Cinema_com_Rapadura','cinemacomrapadura.com.br/'),('Cineclick','www.cineclick.com.br/'), 
('Plano_Critico','www.planocritico.com/'),('Plano_Aberto','www.planoaberto.com.br/'),
('Cinemasim','www.cinemasim.com.br/'),('Omelete','www.omelete.com.br/'),('Jornal_da_Paraiba','www.jornaldaparaiba.com.br/'),
('Cinema_em_Cena','cinemaemcena.cartacapital.com.br/'), ('Elpais','brasil.elpais.com/')]

## Coletando paginas

### BFS

In [3]:
def get_links_bfs(page):
            
    home_page =  page[1]
    print(home_page)
    ##Carrega o robots.txt
    rp = robotparser.RobotFileParser()
    try:
        rp.set_url("https://" +home_page +"robots.txt")
        rp.read()
    except ssl.CertificateError as e:
        rp.set_url("http://" +home_page +"robots.txt")
        rp.read() 
    
    headers = {'user-agent' : 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_5);'}
    #arquivos com backup de paginas visitas e fila de paginas
    with open('lista_paginas.json') as f:
        lista_paginas = json.load(f)
    with open('fila_paginas.json') as f:
        fila_paginas_json = json.load(f)
    
    if len(lista_paginas) == 0:
        lista_paginas = [home_page]
    if len(fila_paginas_json):
        fila_paginas = deque(fila_paginas_json)
        fila_aux = deque(fila_paginas_json)        
    else:
        fila_paginas = deque([home_page])
        fila_aux = deque([home_page])   
    #quantidade de paginas salvas
    cont = 0
    while cont < 1000:
        page_link = fila_paginas.popleft()
              
        try:
            resp = rq.get("https://"+page_link, headers=headers, allow_redirects=False)
        except rq.exceptions.SSLError:
            resp = rq.get("http://"+page_link, headers=headers, allow_redirects=False)
        
        http_encoding = resp.encoding if 'charset' in resp.headers.get('content-type', '').lower() else None
        html_encoding = EncodingDetector.find_declared_encoding(resp.content, is_html=True)
        encoding = html_encoding or http_encoding
        soup = BeautifulSoup(resp.content, from_encoding=encoding)
        links = []
        #verifica se a pagina veio vazia
        if not (resp.content):
            continue
        pattern = re.compile("^(/|http)")
        for link in soup.find_all("a", href=pattern):
            href = link.get('href')
            if href.endswith(".xml") or href.endswith(".jpg") or href.endswith(".png") or href.endswith(".pdf") or href.endswith(".jpeg"):
                continue
            if href.startswith("//"):
                href = href[2:] 
            if href.startswith("/"):
                href = home_page + href[1:]                
            href = href.replace("https://", "")
            href = href.replace("http://", "")
           
            if not (href.startswith(home_page)):
                continue                               
            if href not in lista_paginas:                    
                if cont < 1000:
                    #adiciona na fila se não houver problemas com o robots.txt ou pagina = cinema em cena -não possui robots.txt
                    if rp.can_fetch("*", href) or (page[0] == 'Cinema_em_Cena'):
                        lista_paginas.append(href)
                        fila_paginas.append(href)
                        fila_aux.append(href)
        page_link = page_link.replace("https://", "")
        page_link = page_link.replace("http://", "")
        page_link = page_link.replace("/", "_") 
        cont = cont+1 
        x = fila_aux.popleft()
        with open("pages/bfs/"+page[0]+"/"+page_link+".html", 'wb') as f:
            f.write(resp.content)
        if (cont % 10) == 0:
            time.sleep(randrange(3))
        with open('lista_paginas.json', 'w') as outfile:
            json.dump(lista_paginas, outfile)
        with open('fila_paginas.json', 'w') as outfile:
            json.dump(list(fila_aux), outfile)
    

# with open('lista_paginas.json', 'w') as outfile:
#     json.dump([], outfile)
# with open('fila_paginas.json', 'w') as outfile:
#     json.dump([], outfile)
    
    
#get_links_bfs(paginas[5])



### Heurística

In [4]:
good_words = ["critica","resenha"] 
import ssl
ok_words = ["cultura", "filme","cinema"] 

bad_words = ["noticia", "serie", "lista","eventos","quadrinhos",
             "temporada","album", "literatura", "televisao", "musica", "podcast", "sinopse"] 

index_words = ["/page/", "/index/","page=","/a/"]

def get_link_score(href, text, home_page, pai_score):
    if text == None:
        text = ""
    link = href.replace(home_page, "")
    for word in bad_words:
        if word in link:
            return -1
    for word in good_words:
        if word in link:
            for word in index_words:
                if word in link:
                    return 2
            return 3
    for word in ok_words:
        if word in link:
            return 1
    
    return 0


def get_links_heuristic(page):
     
    home_page =  page[1]
    print(home_page)
    ##Carrega o robots.txt
    rp = robotparser.RobotFileParser()
    try:
        rp.set_url("https://" +home_page +"robots.txt")
        rp.read()
    except ssl.CertificateError as e:
        rp.set_url("http://" +home_page +"robots.txt")
        rp.read()
        
    
    headers = {'user-agent' : 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_5);'}
    #arquivos com backup de paginas visitas e fila de paginas
    with open('lista_paginas.json') as f:
        lista_paginas = json.load(f)
    with open('fila_paginas.json') as f:
        fila_paginas_json = json.load(f)
    with open('cont.json') as f:
        cont = json.load(f)
    
    if len(lista_paginas) == 0:
        lista_paginas = [home_page]
    if len(fila_paginas_json):
        page_ranking = fila_paginas_json
        fila_aux = fila_paginas_json        
    else:
        page_ranking = [(home_page,1)]
        fila_aux = [(home_page,1)]   
    #quantidade de paginas salvas
    last_cont = 0
    not_finding = 0
    while cont < 1000:
        pagina = page_ranking.pop(0)
        page_link = pagina[0]
        page_score = pagina[1]
        #print(page_score)
        try:
            resp = rq.get("https://"+page_link, headers=headers, allow_redirects=False)
        except rq.exceptions.SSLError:
            resp = rq.get("http://"+page_link, headers=headers, allow_redirects=False)
        
        http_encoding = resp.encoding if 'charset' in resp.headers.get('content-type', '').lower() else None
        html_encoding = EncodingDetector.find_declared_encoding(resp.content, is_html=True)
        encoding = html_encoding or http_encoding
        soup = BeautifulSoup(resp.content, from_encoding=encoding)
        links = []
        #verifica se a pagina veio vazia        
        x = fila_aux.pop(0)
        if not (resp.content):
            continue 
        
        pattern = re.compile("^(/|http)")
        for link in soup.find_all("a", href=pattern):
            href = link.get('href')
            text = link.string
            if href.endswith(".xml") or href.endswith(".jpg") or href.endswith(".png") or href.endswith(".pdf") or href.endswith(".jpeg"):
                continue            
            if "#" in href:
                continue
            if href.startswith("//"):
                href = href[2:] 
            if href.startswith("/"):
                href = home_page + href[1:]                
            href = href.replace("https://", "")
            href = href.replace("http://", "")
           
            if not (href.startswith(home_page)):
                continue                               
            if href not in lista_paginas:                    
                if cont < 1000:
                    #adiciona na fila se não houver problemas com o robots.txt ou pagina = cinema em cena -não possui robots.txt
                    if rp.can_fetch("*", href) or (page[0] == 'Plano_Critico'):
                        score = get_link_score(href,text, home_page, page_score)                        
                        page_ranking.append((href,score))
                        page_ranking.sort(key=lambda x: x[1], reverse=True)
                        #print(page_ranking)
                        fila_aux = page_ranking
            lista_paginas.append(href)            
                        
        page_link = page_link.replace("https://", "")
        page_link = page_link.replace("http://", "")
        page_link = page_link.replace("/", "_")
        cont = cont+1 
        if (page_score >= 2):                      
            with open("pages/heuristic/"+page[0]+"/"+page_link+".html", 'wb') as f:
                f.write(resp.content)
        
       
        if (cont % 10) == 0:
            time.sleep(randrange(3))
        with open('lista_paginas.json', 'w') as outfile:
            json.dump(lista_paginas, outfile)
        with open('fila_paginas.json', 'w') as outfile:
            json.dump(fila_aux, outfile)
        with open('cont.json', 'w') as outfile:
            json.dump(cont, outfile)
    

# with open('lista_paginas.json', 'w') as outfile:
#     json.dump([], outfile)
# with open('fila_paginas.json', 'w') as outfile:
#     json.dump([], outfile)
# with open('cont.json', 'w') as outfile:
#     json.dump(0, outfile)

# get_links_heuristic(paginas[0])


In [17]:
good_words = ["critica","resenha", "/cultura/cinema/"] 
import ssl
ok_words = ["cultura", "filme","cinema","cine"] 

bad_words = ["noticia", "serie", "lista","eventos","quadrinhos","governo", "tag/discos/","share=","vida_urbana","colunas/",
             "politica","actualidad", "cultura/agenda", "tag/fecha/","economia","internacional","esportes/","tv/",
             "temporada","album", "literatura", "televisao","critica_literaria/","libros","premios","resenha-de-livro",
             "musica", "podcast", "sinopse", "clipe","critica_arte/", "autor/", "emmy","tag/game_of_thrones" ] 

index_words = ["/page/", "/index/","page=","/a/"]

def get_link_score_2(href, text, home_page, pai_score):
    regexp = re.compile('[0-9]x[0-9]')
    
    link = href.replace(home_page, "")
    if regexp.search(link):
        return -1    
    for word in bad_words:
        if word in link:
            return -1
    for word in good_words:
        if word in link:            
            for word in index_words:
                if word in link:
                    #print(href)
                    return 3
            return 10
    for word in ok_words:
        if word in link:
            if pai_score == 3:
                return 4
            return 1
    if pai_score == 3:
                #print(href)
                return 2
    return 0


def get_links_heuristic_2(page):
     
    home_page =  page[1]
    
    ##Carrega o robots.txt
    rp = robotparser.RobotFileParser()
    try:
        rp.set_url("https://" +home_page +"robots.txt")
        rp.read()
    except ssl.CertificateError as e:
        rp.set_url("http://" +home_page +"robots.txt")
        rp.read()
    print(home_page)    
    
    headers = {'user-agent' : 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_5);'}
    #arquivos com backup de paginas visitas e fila de paginas
    with open('lista_paginas.json') as f:
        lista_paginas = json.load(f)
    with open('fila_paginas.json') as f:
        fila_paginas_json = json.load(f)
    with open('cont.json') as f:
        cont = json.load(f)
    
    if len(lista_paginas) == 0:
        lista_paginas = [home_page]
    if len(fila_paginas_json):
        page_ranking = fila_paginas_json
        fila_aux = fila_paginas_json        
    else:
        page_ranking = [(home_page,1)]
        fila_aux = [(home_page,1)]   
    #quantidade de paginas salvas
    last_cont = cont
    not_finding = 0
    while cont < 1000:
        pagina = page_ranking.pop(0)
        page_link = pagina[0] 
        page_score = pagina[1]
        #print(page_link, page_score)
        try:
            resp = rq.get("https://"+page_link, headers=headers, allow_redirects=False)
        except rq.exceptions.SSLError:
            resp = rq.get("http://"+page_link, headers=headers, allow_redirects=False)
        
        http_encoding = resp.encoding if 'charset' in resp.headers.get('content-type', '').lower() else None
        html_encoding = EncodingDetector.find_declared_encoding(resp.content, is_html=True)
        encoding = html_encoding or http_encoding
        soup = BeautifulSoup(resp.content, from_encoding=encoding)
        links = []
        #verifica se a pagina veio vazia        
        x = fila_aux.pop(0)
        if not (resp.content):
            continue 
        
        pattern = re.compile("^(/|http)")
        for link in soup.find_all("a", href=pattern):
            href = link.get('href')
            
            text = link.string
            if href.endswith(".xml") or href.endswith(".jpg") or href.endswith(".png") or href.endswith(".pdf") or href.endswith(".jpeg"):
                continue            
            if "#" in href:
                continue
            if href.startswith("//"):
                href = href[2:] 
            if href.startswith("/"):
                href = home_page + href[1:]                
            href = href.replace("https://", "")
            href = href.replace("http://", "")
            
            if not (href.startswith(home_page)):
                continue
           
            if href not in lista_paginas:                
                if cont < 1000:
                    #print(href)
                    #adiciona na fila se não houver problemas com o robots.txt ou pagina = cinema em cena -não possui robots.txt
                    if rp.can_fetch("*", href) or (page[0] == 'Cinema_em_Cena'):
                        
                        score = get_link_score_2(href,text, home_page, page_score)                        
                        page_ranking.append((href,score))
                        page_ranking.sort(key=lambda x: x[1], reverse=True)                        
                        
                        fila_aux.append((href,score))
                        fila_aux.sort(key=lambda x: x[1], reverse=True)
            lista_paginas.append(href)            
            
        page_link = page_link.replace("https://", "")
        page_link = page_link.replace("http://", "")
        page_link = page_link.replace("/", "_")
        cont = cont+1 
        if (page_score >= 2 and page_score != 3):
            last_cont = cont
            with open("pages/heuristic_2/"+page[0]+"/"+page_link+".html", 'wb') as f:
                f.write(resp.content)       
        
        if (cont % 10) == 0:
            time.sleep(randrange(3))
        with open('lista_paginas.json', 'w') as outfile:
            json.dump(lista_paginas, outfile)
        with open('fila_paginas.json', 'w') as outfile:
            json.dump(fila_aux, outfile)
        with open('cont.json', 'w') as outfile:
            json.dump(cont, outfile)
            
        if(cont - last_cont) > 30:
            print("muito tempo ocioso")
            break
        if(len(page_ranking) == 0):
            print("fila vazia")
            break
    

with open('lista_paginas.json', 'w') as outfile:
    json.dump([], outfile)
with open('fila_paginas.json', 'w') as outfile:
    json.dump([], outfile)
with open('cont.json', 'w') as outfile:
    json.dump(0, outfile)

get_links_heuristic_2(paginas[2])


www.cineclick.com.br/


/home/leonardo/anaconda3/lib/python3.6/site-packages/bs4/__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 193 of the file /home/leonardo/anaconda3/lib/python3.6/runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "lxml")

  markup_type=markup_type))
